<a href="https://colab.research.google.com/github/Adithyan773/Google-colab-models/blob/main/SMS_spam_detection_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets
import opendatasets as od
import pandas as pd

In [2]:
od.download("https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: adithyankm
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset


100%|██████████| 211k/211k [00:00<00:00, 401kB/s]

In [3]:
df = pd.read_csv("/content/sms-spam-collection-dataset/spam.csv",encoding='latin-1')

In [4]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [6]:
df.drop(columns = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace = True)

In [7]:
counts = df['v1'].value_counts()
counts

,count
v1,
ham,4825
spam,747


In [8]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['v1'] = encoder.fit_transform(df['v1'])

In [9]:
df.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [10]:
df.isnull().sum()

,0
v1,0
v2,0


In [11]:
df = df.drop_duplicates(keep = 'first')

In [12]:
import nltk
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

In [13]:
import string,re,nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [14]:
stopwords = nltk.corpus.stopwords.words('english')
porter_stemmer = PorterStemmer()

In [15]:
string.punctuation
def remove_punctuations(v2):
  for punctuation in string.punctuation:
    v2 = v2.replace(punctuation,'')
  return v2
def tokenization(text):
    tokens = re.split(' ',text)
    return tokens
def remove_stopwords(text):
  stopwords_set = set(stopwords)
  output = [i for i in text if i not in stopwords_set]
  return output
def stemming(text):
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text
def lemmatizer(text):
  lemmatizer = WordNetLemmatizer()
  lemm_text = [lemmatizer.lemmatize(word) for word in text]
  return lemm_text
def remove_numbers(text_list):
  text = ' '.join(text_list)
  text = re.sub(r'\d+', '', text)
  return text.split()

In [16]:
df['clean_msg']= df['v2'].apply(lambda x:remove_punctuations(x))
df['msg_lower']= df['clean_msg'].apply(lambda x: x.lower())
df['msg_tokenized']= df['msg_lower'].apply(lambda x: tokenization(x))
df['no_stopwords']= df['msg_tokenized'].apply(lambda x:remove_stopwords(x))
df['stemmed']= df['no_stopwords'].apply(lambda x: stemming(x))
df['numbers_removed']= df['stemmed'].apply(lambda x: remove_numbers(x))
df['lemmatized']= df['no_stopwords'].apply(lambda x:lemmatizer(x))

In [17]:
documents = []
documents.extend(df['numbers_removed'])
for i in range(len(documents)):
  documents[i] = ' '.join(documents[i])

In [18]:
documents[0]

'go jurong point crazi avail bugi n great world la e buffet cine got amor wat'

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7)
X = tfidfconverter.fit_transform(documents).toarray()

In [20]:
tfidfconverter.get_feature_names_out()[:150]

array(['abiola', 'abl', 'abt', 'access', 'account', 'across', 'activ',
       'actual', 'ad', 'add', 'address', 'admir', 'ador', 'adult',
       'advic', 'affair', 'aft', 'afternoon', 'aftr', 'age', 'ago', 'ah',
       'aha', 'ahead', 'ahmad', 'aight', 'aint', 'air', 'airport',
       'aiyah', 'aiyo', 'al', 'alert', 'alex', 'allow', 'almost', 'alon',
       'alreadi', 'alright', 'alrit', 'also', 'alway', 'am', 'amount',
       'amp', 'ampm', 'an', 'angri', 'announc', 'anoth', 'answer',
       'anybodi', 'anymor', 'anyon', 'anyth', 'anytim', 'anyway', 'apart',
       'app', 'appli', 'appoint', 'appreci', 'april', 'ar', 'ard', 'area',
       'arent', 'arm', 'armand', 'around', 'arrang', 'arriv', 'art',
       'asap', 'ask', 'askd', 'askin', 'asleep', 'ass', 'assum', 'ate',
       'attempt', 'attend', 'auction', 'aunti', 'avail', 'ave', 'avoid',
       'await', 'awak', 'award', 'away', 'awesom', 'babe', 'babi', 'back',
       'bad', 'bag', 'bak', 'balanc', 'bank', 'bar', 'bare', 'basic',


In [21]:
y = df['v1']

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [23]:
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:",accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

Accuracy: 0.97678916827853
[[883   2]
 [ 22 127]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       885
           1       0.98      0.85      0.91       149

    accuracy                           0.98      1034
   macro avg       0.98      0.93      0.95      1034
weighted avg       0.98      0.98      0.98      1034

